In [1]:
from pathlib import Path

import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms

import searchnets
from searchnets.utils.dataset import VisSearchDataset

In [2]:
'''LeNet in PyTorch.'''

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*53*53, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 2)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [3]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

normalize = transforms.Normalize(mean=MEAN,
                                 std=STD)

batch_size = 64
num_workers = 32

def get_train_and_val(csv_file):
    trainset = VisSearchDataset(csv_file=csv_file,
                            split='train',
                            transform=transforms.Compose(
                                [transforms.ToTensor(), normalize]
                            ))

    train_loader = DataLoader(trainset, batch_size=batch_size,
                              shuffle=True, num_workers=num_workers,
                              pin_memory=True)

    valset = VisSearchDataset(csv_file=csv_file,
                              split='val',
                              transform=transforms.Compose([transforms.ToTensor(), normalize]))
    val_loader = DataLoader(valset, batch_size=batch_size,
                            shuffle=False, num_workers=num_workers)
    
    return trainset, train_loader, valset, val_loader

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
def train_one_epoch(model, trainset, train_loader, criterion, optimizer):
    """train model for one epoch"""
    model.train()

    total_loss = 0.0

    batch_total = int(np.ceil(len(trainset) / batch_size))
    batch_pbar = tqdm(train_loader)
    for i, (batch_x, batch_y) in enumerate(batch_pbar):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        output = model(batch_x)
        loss = criterion(output, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_pbar.set_description(f'batch {i} of {batch_total}, loss: {loss: 7.3f}')
        total_loss += loss

    avg_loss = total_loss / batch_total
    print(f'\tTraining Avg. Loss: {avg_loss:7.3f}')
    return avg_loss

In [6]:
def validate(model, valset, val_loader):
    model.eval()

    val_acc_this_epoch = []
    with torch.no_grad():
        total = int(np.ceil(len(valset) / batch_size))
        pbar = tqdm(val_loader)
        for i, (batch_x, batch_y) in enumerate(pbar):
            pbar.set_description(f'batch {i} of {total}')
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            output = model(batch_x)
            # below, _ because torch.max returns (values, indices)
            _, predicted = torch.max(output.data, 1)
            acc = (predicted == batch_y).sum().item() / batch_y.size(0)
            val_acc_this_epoch.append(acc)

    val_acc_this_epoch = np.asarray(val_acc_this_epoch).mean()
    print(' Validation Acc: %7.3f' % val_acc_this_epoch)

    return val_acc_this_epoch

In [7]:
epochs = 200
val_epoch = 1
patience = 10

In [8]:
def train(csv_file):
    trainset, train_loader, valset, val_loader = get_train_and_val(csv_file)

    model = LeNet()
    model.to(device);

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(),
                                lr=0.001,
                                momentum=0.9)
    
    loss = []
    best_val_acc = 0
    epochs_without_improvement = 0

    for epoch in range(1, epochs + 1):

        print(f'\nEpoch {epoch}')
        avg_loss = train_one_epoch(model, trainset, train_loader, criterion, optimizer)
        loss.append(avg_loss)

        if epoch % val_epoch == 0:
            val_acc_this_epoch = validate(model, valset, val_loader)

            if patience is not None:
                if val_acc_this_epoch > best_val_acc:
                    best_val_acc = val_acc_this_epoch
                    epochs_without_improvement = 0
                    print(f'Validation accuracy improved')
                else:
                    epochs_without_improvement += 1
                    if epochs_without_improvement > patience:
                        print(
                            f'greater than {patience} epochs without improvement in validation '
                            'accuracy, stopping training')

                        break

    return model, loss

In [9]:
def get_test(csv_file):
        testset = VisSearchDataset(csv_file=csv_file,
                                    split='test',
                                    transform=transforms.Compose(
                                        [transforms.ToTensor(), normalize]
                                    ))
        test_loader = DataLoader(testset, batch_size=batch_size,
                                 shuffle=False, num_workers=num_workers,
                                 pin_memory=True)
        return testset, test_loader

In [10]:
def test(model, csv_file):
    testset, test_loader = get_test(csv_file)
    model.eval()

    total = int(np.ceil(len(testset) / batch_size))
    pbar = tqdm(test_loader)
    acc = []
    pred = []
    with torch.no_grad():
        for i, (x_batch, y_batch) in enumerate(pbar):
            pbar.set_description(f'batch {i} of {total}')
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            output = model(x_batch)
            # below, _ because torch.max returns (values, indices)
            _, pred_batch = torch.max(output.data, 1)
            acc_batch = (pred_batch == y_batch).sum().item() / y_batch.size(0)
            acc.append(acc_batch)

    acc = np.asarray(acc).mean()

    return acc

In [11]:
csv_files = [
    # '~/Documents/repos/L2M/visual-search-nets/data/visual_search_stimuli/alexnet_RVvGV/alexnet_RVvGV_finetune_split.csv',
    '~/Documents/repos/L2M/visual-search-nets/data/visual_search_stimuli/alexnet_RVvRHGV/alexnet_RVvRHGV_finetune_split.csv',
    '~//Documents/repos/L2M/visual-search-nets/data/visual_search_stimuli/alexnet_2_v_5/alexnet_2_v_5_finetune_split.csv',
]

In [12]:
for csv_file in csv_files:
    csv_file = Path(csv_file)
    csv_file = csv_file.expanduser()
    model, loss_history = train(csv_file)
    acc = test(model, csv_file)
    print(f'\taccuracy on test set: {acc}')

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1


  0%|          | 0/4 [00:00<?, ?it/s]

	Training Avg. Loss:   0.693


  0%|          | 0/100 [00:00<?, ?it/s]

...

	Training Avg. Loss:   0.000


  0%|          | 0/13 [00:00<?, ?it/s]

 Validation Acc:   0.996
greater than 10 epochs without improvement in validation accuracy, stopping training


batch 12 of 13: 100%|██████████| 13/13 [00:04<00:00,  1.39s/it]

	accuracy on test set: 0.9975961538461539
